In [2]:
import torch, os, cv2
from utils.dist_utils import dist_print
import torch, os
from utils.common import merge_config, get_model
from tqdm import tqdm
import torchvision.transforms as transforms
from data.dataset import LaneTestDataset
from utils.config import Config
import numpy as np
from PIL import Image


/home/czh/anaconda3/envs/lane-det/lib/python3.7/site-packages/nvidia/dali/backend.py:57: Warning: DALI 1.31 is the last release to support Python 3.7. Please update your environment to use Python 3.8, 3.9, 3.10, or (experimentally) 3.11.
  deprecation_warning("DALI 1.31 is the last release to support Python 3.7. "


In [19]:
def pred2coords(pred, row_anchor, col_anchor, local_width = 1, original_image_width = 1640, original_image_height = 590):
    batch_size, num_grid_row, num_cls_row, num_lane_row = pred['loc_row'].shape
    batch_size, num_grid_col, num_cls_col, num_lane_col = pred['loc_col'].shape

    max_indices_row = pred['loc_row'].argmax(1).cpu()
    # n , num_cls, num_lanes
    valid_row = pred['exist_row'].argmax(1).cpu()
    # n, num_cls, num_lanes

    max_indices_col = pred['loc_col'].argmax(1).cpu()
    # n , num_cls, num_lanes
    valid_col = pred['exist_col'].argmax(1).cpu()
    # n, num_cls, num_lanes

    pred['loc_row'] = pred['loc_row'].cpu()
    pred['loc_col'] = pred['loc_col'].cpu()

    coords = []

    row_lane_idx = [1,2]
    col_lane_idx = [0,3]

    for i in row_lane_idx:
        tmp = []
        if valid_row[0,:,i].sum() > num_cls_row / 2:
            for k in range(valid_row.shape[1]):
                if valid_row[0,k,i]:
                    all_ind = torch.tensor(list(range(max(0,max_indices_row[0,k,i] - local_width), min(num_grid_row-1, max_indices_row[0,k,i] + local_width) + 1)))
                    
                    out_tmp = (pred['loc_row'][0,all_ind,k,i].softmax(0) * all_ind.float()).sum() + 0.5
                    out_tmp = out_tmp / (num_grid_row-1) * original_image_width
                    tmp.append((int(out_tmp), int(row_anchor[k] * original_image_height)))
            coords.append(tmp)

    for i in col_lane_idx:
        tmp = []
        if valid_col[0,:,i].sum() > num_cls_col / 4:
            for k in range(valid_col.shape[1]):
                if valid_col[0,k,i]:
                    all_ind = torch.tensor(list(range(max(0,max_indices_col[0,k,i] - local_width), min(num_grid_col-1, max_indices_col[0,k,i] + local_width) + 1)))
                    
                    out_tmp = (pred['loc_col'][0,all_ind,k,i].softmax(0) * all_ind.float()).sum() + 0.5

                    out_tmp = out_tmp / (num_grid_col-1) * original_image_height
                    tmp.append((int(col_anchor[k] * original_image_width), int(out_tmp)))
            coords.append(tmp)

    return coords

In [20]:
torch.backends.cudnn.benchmark = True
batch_size = 1


In [26]:
config_file = './configs/culane_res34.py'
cfg = Config.fromfile(config_file)
net = get_model(cfg)
cfg.test_model = './culane_res34.pth'
cfg.row_anchor = np.linspace(0.4, 1, cfg.num_row)
cfg.col_anchor = np.linspace(0, 1, cfg.num_col)
state_dict = torch.load(cfg.test_model, map_location='cpu')['model']

In [27]:
compatible_state_dict = {}
for k, v in state_dict.items():
    if 'module.' in k:
        compatible_state_dict[k[7:]] = v
    else:
        compatible_state_dict[k] = v

net.load_state_dict(compatible_state_dict, strict=False)
net.eval()

parsingNet(
  (model): resnet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_ru

In [28]:
img_transforms = transforms.Compose([
        transforms.Resize((int(cfg.train_height / cfg.crop_ratio), cfg.train_width)),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ])

In [30]:
folder_list = os.listdir(r'../Roadviewer/Front')
out_path = '../Roadviewer/Test_out'
if not os.path.exists(out_path):
    os.makedirs(out_path)
for folder in folder_list:
    img_folder = os.path.join(r'../Roadviewer/Front', folder)
    image_list = os.listdir(img_folder)
    out_foler = os.path.join(out_path, folder)
    if not os.path.exists(out_foler):
        os.makedirs(out_foler)
    for i,img_path in tqdm(enumerate(image_list)):
        # Read the image
        img = Image.open(os.path.join(img_folder, img_path))
        img_w, img_h = img.size
        # resize the image
        # img = img.resize((img_w,img_h))

        
        # Convert the image to tensor
        img = img_transforms(img)
        img = img[:,-cfg.train_height:,:]
        img = img.unsqueeze(0)
        # Move the image to GPU if available
        img = img.cuda()
        # forward
        with torch.no_grad():
            pred = net(img)
        coords = pred2coords(pred, cfg.row_anchor, cfg.col_anchor, original_image_width = img_w, original_image_height = img_h)
        vis = cv2.imread(os.path.join(img_folder, img_path))
        # resize the image as the original image
        # vis = cv2.resize(vis, (img_w,img_h))
        for lane in coords:
            for coord in lane:
                cv2.circle(vis, coord, 5, (255,0,0), -1)
        cv2.imwrite(os.path.join(out_foler, f'{i}.jpg'), vis)        
    

200it [00:20,  9.92it/s]
198it [00:19, 10.08it/s]
74it [00:07,  9.94it/s]
200it [00:19, 10.08it/s]
200it [00:19, 10.47it/s]
200it [00:20,  9.90it/s]
200it [00:19, 10.46it/s]
200it [00:19, 10.15it/s]
200it [00:19, 10.30it/s]
